# Representações Latentes e Autoencoders

Neste notebook, exploraremos o conceito de representações latentes e como os autoencoders podem ser utilizados para aprendê-las de forma não supervisionada. Abordaremos os seguintes tópicos:

* **Representações Latentes**:
    * Definição formal e o objetivo de aprender representações de dados em espaços de menor dimensionalidade.
    * Visualização do espaço latente com o algoritmo t-SNE (t-Distributed Stochastic Neighbor Embedding).
    * Exemplo prático de treinamento de um codificador (encoder) para a tarefa de classificação e a subsequente visualização de suas representações latentes para o dataset MNIST.

* **Autoencoders**:
    * Arquitetura fundamental de um autoencoder (encoder-decoder).
    * Implementação e treinamento de um autoencoder para reconstruir imagens do MNIST.
    * Visualização de imagens originais e reconstruídas.

* **Denoising Autoencoders**:
    * Conceito e formulação de autoencoders com remoção de ruído.
    * Aplicação prática na remoção de ruído de imagens do MNIST.

## Representações Latentes

No contexto de Deep Learning, uma representação latente é uma codificação interna dos dados de entrada que emerge como resultado do processo de treinamento de uma rede neural para uma tarefa específica. Em vez de ser um resultado de um algoritmo de redução de dimensionalidade pré-definido, o espaço latente é aprendido dinamicamente.

As redes neurais profundas são compostas por uma sequência de camadas. Cada camada executa uma transformação sobre seu dado de entrada, e o resultado é uma representação progressivamente mais abstrata. A saída de qualquer camada intermediária de uma rede pode ser considerada uma representação latente.

A estrutura e as propriedades deste espaço latente são diretamente influenciadas pela função objetivo (loss function) que o modelo otimiza.

Em Aprendizagem Supervisionada (e.g., Classificação), o modelo é treinado para minimizar um erro de classificação (como a Entropia Cruzada). Para isso, o algoritmo de backpropagation ajusta os pesos da rede de forma a transformar os dados de entrada em representações internas que tornem as classes o mais separáveis possível. Um espaço latente ideal, neste caso, agrupará amostras da mesma classe em regiões coesas e distintas, idealmente permitindo uma separação linear por parte das camadas finais da rede. A representação aprende a reter apenas as características discriminativas para a tarefa.

Formalmente, o mapeamento para o espaço latente é uma função parametrizada $\mathbf{z} = f_{\theta}(x)$, onde $\theta$ representa os pesos da rede (o *encoder*) e $\mathbf{z} \in \mathbb{R}^m$ é o vetor latente. Esses parâmetros $\theta$ são aprendidos através da otimização de gradiente descendente para minimizar a função de perda da tarefa final.

In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms

# Transformação
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])

# Datasets
train_data = datasets.MNIST("./data", train=True, download=True, transform=transform)
test_data  = datasets.MNIST("./data", train=False, download=True, transform=transform)

# Subsets menores
train_subset = Subset(train_data, torch.randperm(len(train_data))[:10000])
val_subset   = Subset(test_data,  torch.randperm(len(test_data))[:1000])

# DataLoaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_subset, batch_size=64)

print(len(train_subset), len(val_subset))

## Autoencoders

Um Autoencoder é um tipo de rede neural artificial utilizada para aprender representações de dados de forma não supervisionada. A arquitetura é composta por duas partes principais:

1.  **Encoder ($f$)**: Mapeia a entrada $x$ para uma representação latente $z$ de menor dimensão. $z = f(x)$.
2.  **Decoder ($g$)**: Tenta reconstruir a entrada original a partir da representação latente $z$. $\hat{x} = g(z)$.

O objetivo do treinamento é minimizar o erro de reconstrução, que é a diferença entre a entrada original $x$ e a sua reconstrução $\hat{x}$. Uma função de perda comum para essa tarefa é o Erro Quadrático Médio (Mean Squared Error - MSE).

$$ \mathcal{L}(x, \hat{x}) = \mathcal{L}(x, g(f(x))) = \frac{1}{N} \sum_{i=1}^{N} (x_i - \hat{x}_i)^2 $$

O gargalo informacional imposto pela camada latente força o autoencoder a aprender apenas as variações mais importantes nos dados.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim=16):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, encoding_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
latent_dim = 16
ae_model = Autoencoder(encoding_dim=latent_dim).to(device)
print(ae_model)

In [ ]:
ae_criterion = nn.MSELoss()
ae_optimizer = torch.optim.Adam(ae_model.parameters(), lr=1e-3, weight_decay=1e-8)

In [ ]:
epochs = 20
ae_model.train()

train_losses = []

for epoch in range(epochs):
    epoch_loss = 0.0
    for X, _ in train_loader:
        X = X.view(X.size(0), -1).to(device)
        
        # Forward
        recon = ae_model(X)
        loss = ae_criterion(recon, X)

        # Backward
        ae_optimizer.zero_grad()
        loss.backward()
        ae_optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


plt.figure(figsize=(8,5))
plt.plot(range(1, epochs+1), train_losses, marker="o")
plt.title("Training Loss (Autoencoder)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

### Visualização das Reconstruções

Após o treinamento, podemos passar imagens do conjunto de teste pelo autoencoder para obter suas reconstruções. Comparar visualmente as imagens originais com as reconstruídas nos dá uma avaliação qualitativa do desempenho do modelo.

In [ ]:
# Visualização das imagens reconstruídas
ae_model.eval()
n = 10

with torch.no_grad():
    # Pega um batch de dados de validação
    data_iter = iter(val_loader)
    images, _ = next(data_iter)
    images_flat = images.view(images.size(0), -1).to(device)
    
    # Gera as reconstruções
    reconstructed_flat = ae_model(images_flat)
    
    # Converte para numpy para plotar
    original_images = images.cpu().numpy()
    reconstructed_images = reconstructed_flat.view(-1, 1, 28, 28).cpu().numpy()

In [ ]:
plt.figure(figsize=(20, 4))

for i in range(n):
    # Imagem original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(original_images[i].squeeze(), cmap='gray', vmin=0, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Imagens Originais')

    # Imagem reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(reconstructed_images[i].squeeze(), cmap='gray', vmin=0, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Imagens Reconstruídas')

plt.show()

In [ ]:
ae_model.eval()
ae_all_latents = []
ae_all_labels = []

with torch.no_grad():
    for X, y in val_loader:
        X = X.view(X.size(0), -1).to(device)
        latent = ae_model.encoder(X)
        ae_all_latents.append(latent.cpu().numpy())
        ae_all_labels.append(y.cpu().numpy())

ae_latent_space_test = np.concatenate(ae_all_latents, axis=0)
ae_labels_test = np.concatenate(ae_all_labels, axis=0)

tsne = TSNE(n_components=2, random_state=42, verbose=1, perplexity=30, n_iter=1000)
tsne_results = tsne.fit_transform(ae_latent_space_test)

plt.figure(figsize=(12, 10))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=ae_labels_test, cmap='tab10', s=10)
plt.title('Visualização do Espaço Latente com t-SNE', fontsize=16)
plt.xlabel('Componente t-SNE 1')
plt.ylabel('Componente t-SNE 2')
plt.legend(handles=scatter.legend_elements()[0], labels=list(range(10)))
plt.grid(True)
plt.show()

## Interpolando no espaço latente

A partir das representações latentes aprendidas pelo encoder, podemos explorar o espaço latente realizando **interpolação linear** entre dois vetores $z_1$ e $z_2$. Essa técnica serve para verificar se o autoencoder aprendeu um espaço contínuo e semanticamente estruturado, no qual pequenas mudanças em $z$ resultam em mudanças suaves nas reconstruções.

Dada uma interpolação linear:

$$
z(t) = (1 - t) z_1 + t z_2,\quad t \in [0, 1],
$$

podemos decodificar cada ponto intermediário e visualizar a transição entre duas imagens reais.

In [ ]:
# Selecionar duas imagens do conjunto de validação
ae_model.eval()

with torch.no_grad():
    imgs, _ = next(iter(val_loader))
    x1 = imgs[0].view(1, -1).to(device)
    x2 = imgs[1].view(1, -1).to(device)

    z1 = ae_model.encoder(x1)
    z2 = ae_model.encoder(x2)

# Interpolar entre z1 e z2
steps = 12
interpolations = []

with torch.no_grad():
    for t in np.linspace(0, 1, steps):
        zt = (1 - t) * z1 + t * z2
        xt = ae_model.decoder(zt).view(28, 28).cpu().numpy()
        interpolations.append(xt)

# Plot
plt.figure(figsize=(20, 3))
for i, img in enumerate(interpolations):
    ax = plt.subplot(1, steps, i + 1)
    plt.imshow(img, cmap="gray", vmin=0, vmax=1)
    ax.axis("off")
plt.suptitle("Interpolação Linear no Espaço Latente")
plt.show()

## Gerando novos exemplos

Após o treinamento, o decoder pode ser usado isoladamente para gerar novas imagens a partir de vetores latentes amostrados manualmente. Como o espaço latente do autoencoder não é regularizado, essas amostras não seguem uma distribuição específica; ainda assim, podemos testar valores aleatórios em $\mathbb{R}^m$ e observar as reconstruções produzidas.

O procedimento é simples:
1. Amostre um vetor aleatório $z$ (por exemplo, de uma distribuição normal padrão).
2. Passe $z$ pelo decoder.
3. Visualize o resultado como uma imagem de 28×28 pixels.

Isso nos permite avaliar o quanto o modelo aprendeu a estrutura dos dígitos no espaço latente.

In [ ]:
# Gerar novas imagens a partir de vetores latentes aleatórios
ae_model.eval()

num_samples = 12
rand_z = torch.randn(num_samples, latent_dim).to(device)

with torch.no_grad():
    generated = ae_model.decoder(rand_z).view(-1, 28, 28).cpu().numpy()

plt.figure(figsize=(20, 3))
for i in range(num_samples):
    ax = plt.subplot(1, num_samples, i + 1)
    plt.imshow(generated[i], cmap='gray', vmin=0, vmax=1)
    ax.axis("off")
plt.suptitle("Imagens Geradas a Partir de Amostras Aleatórias do Latent Space")
plt.show()

### Por quê a geração é problemática?

Autoencoders tradicionais não impõem nenhuma estrutura probabilística ao espaço latente. Isso cria vários problemas:

* O espaço latente não segue uma distribuição simples (por exemplo, uma normal isotrópica).  
* Regiões do espaço latente podem não corresponder a imagens válidas.  
* Amostrar $z$ aleatoriamente leva o decoder a áreas que ele nunca viu durante o treinamento.  
* O modelo não aprende um mapeamento explícito entre uma distribuição simples e a distribuição dos dados.

Como consequência, a geração tende a produzir imagens distorcidas ou incoerentes. Esse é exatamente o ponto em que entram os **Variational Autoencoders (VAEs)**: eles forçam o espaço latente a seguir uma distribuição bem definida, permitindo geração consistente e controlada.